In [ ]:
import h5py
import numpy as np
import torch
import os

In [ ]:
folder = "individual_predictions"
last = 5
files_3=sorted([x for x in os.listdir(folder) if ('FINAL-THREE' in x)])[-last:]
files_5=sorted([x for x in os.listdir(folder) if ('FINAL-FIVE' in x)])[-last:]
files_6=sorted([x for x in os.listdir(folder) if ('FINAL-SIX' in x)])[-last:]
files_7=sorted([x for x in os.listdir(folder) if ('FINAL-SEVEN' in x)])[-last:]
files_8=sorted([x for x in os.listdir(folder) if ('FINAL-EIGHT' in x)])[-last:]
files_10=sorted([x for x in os.listdir(folder) if ('FINAL-TEN' in x)])[-last:]
files_11=sorted([ x  for x in os.listdir(folder) if ('FINAL-ELEVEN' in  x)])[-last:]
files_12=sorted([ x  for x in os.listdir(folder) if ('FINAL-TWELVE' in  x)])[-last:]

In [ ]:
files_3=[os.path.join(folder,s) for s in files_3]
files_5=[os.path.join(folder,s) for s in files_5]
files_6=[os.path.join(folder,s) for s in files_6]
files_7=[os.path.join(folder,s) for s in files_7]
files_8=[os.path.join(folder,s) for s in files_8]
files_10=[os.path.join(folder,s) for s in files_10]
files_11=[os.path.join(folder,s) for s in files_11]
files_12=[os.path.join(folder,s) for s in files_12]

In [ ]:
def ens_from_files(files):
    ens=dict()
    for file in files:
        with h5py.File(file) as d:
            for timestamp in d.keys():
                if timestamp not in ens:
                    ens[timestamp]=0
                ens[timestamp]+=np.array(d[timestamp]['intensity']).astype(np.float64)
    for key in ens.keys():
        ens[key]=ens[key]/len(files)
    return ens

In [ ]:
ens_3=ens_from_files(files_3)
ens_5=ens_from_files(files_5)
ens_6=ens_from_files(files_6)
ens_7=ens_from_files(files_7)
ens_8=ens_from_files(files_8)
ens_10=ens_from_files(files_10)
ens_11=ens_from_files(files_11)
ens_12=ens_from_files(files_12)

In [ ]:
def average_ens(ens_list):
    final=dict()
    for key in ens_list[0].keys():
        final[key]=0
    for ens in ens_list:
        for key in final.keys():
            final[key]+=ens[key]
    for key in final.keys():
        final[key]=final[key]/len(ens_list)
    return final

In [ ]:
ans1=average_ens([ens_3,ens_6,ens_7,ens_8,ens_10])
ans2=average_ens([ens_5,ens_11,ans1])
final = average_ens([ens_12,ans2])

In [ ]:
output_file = "final_output.hdf5"
with h5py.File(output_file, mode='a') as f_out:
    for item in final.items():
        f_out.create_group(item[0])
        f_out[item[0]].create_dataset(
            'intensity',
            data=item[1].astype(np.float32)
        )